In [1]:
'''
导入模型评估包
'''
from datasets import load_metric
from sentence_transformers import SentenceTransformer, util
from transformers import AutoProcessor, CLIPModel, AutoTokenizer
import pickle
import torch

ModuleNotFoundError: No module named 'aiohttp'

In [10]:
# 相关变量和参数
subj = 'subj01'
test_captions= []
captions_from_images = []
captions_from_brain = []

In [18]:
'''加载captions'''
# 人工标注字幕(真实标签)
with open(f"decoded_captions/{subj}/test_captions.sav","rb") as f:
    test_captions = pickle.load(f)

# 图像字幕
with open(f"decoded_captions/{subj}/image_captions.sav","rb") as f:
    captions_from_images = pickle.load(f) 

# 大脑字幕
with open(f"decoded_captions/{subj}/brain_captions.sav","rb") as f:
    captions_from_brain = pickle.load(f)
    

In [20]:
'''  METEOR 评估指标 '''
meteor = load_metric('meteor')

meteor_img_ref=meteor.compute(predictions=captions_from_images,references=test_captions)
# meteor_brain_ref=meteor.compute(predictions=captions_from_brain,references=test_captions)
meteor_brain_img=meteor.compute(predictions=captions_from_brain,references=captions_from_images)

# relative_brain_image_meteor=meteor_brain_img["meteor"]/meteor_img_ref["meteor"]

print(f"[GROUND] METEOR GIT from images vs captions: {meteor_img_ref['meteor']}")
# print(f"[ABSOLUTE] METEOR GIT from brain vs captions: {meteor_brain_ref['meteor']}")
print(f"[ABSOLUTE] METEOR GIT from brain vs images: {meteor_brain_img['meteor']}")

# print(f"[RELATIVE] METEOR  {relative_brain_image_meteor}")

/data/zlhu/miniconda3/envs/ldm2/lib/python3.8/site-packages/datasets/load.py:759: FutureWarning: The repository for meteor contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/meteor/meteor.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
[nltk_data] Downloading package wordnet to /data/zlhu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /data/zlhu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /data/zlhu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


[GROUND] METEOR GIT from images vs captions: 0.4042709206426454
[ABSOLUTE] METEOR GIT from brain vs images: 0.30495808381140255


In [24]:
''' sentence transformer 评估指标 '''
sentence_model = SentenceTransformer('./sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
with torch.no_grad():
    embedding_brain= sentence_model.encode(captions_from_brain, convert_to_tensor=True)
    embedding_captions = sentence_model.encode(test_captions, convert_to_tensor=True)
    embedding_images = sentence_model.encode(captions_from_images, convert_to_tensor=True)

    ss_sim_brain_img=util.pytorch_cos_sim(embedding_brain, embedding_images).cpu()
    ss_sim_brain_cap=util.pytorch_cos_sim(embedding_brain, embedding_captions).cpu()
    ss_sim_img_cap=util.pytorch_cos_sim(embedding_images, embedding_captions).cpu()

    # relative_brain_image_ss=ss_sim_brain_img.diag().mean()/ss_sim_img_cap.diag().mean()


print(f"[GROUND] Sentence Transformer Similarity GIT from image vs human: {ss_sim_img_cap.diag().mean()}")
print(f"[ABSOLUTE] Sentence Transformer Similarity GIT from brain vs image: {ss_sim_brain_img.diag().mean()}")
# print(f"[RELATIVE] Sentence Transformer Similarity   {relative_brain_image_ss.mean()}")

[GROUND] Sentence Transformer Similarity GIT from image vs human: 0.7029381394386292
[ABSOLUTE] Sentence Transformer Similarity GIT from brain vs image: 0.4474000036716461


In [25]:
''' clip similarity 评估指标 '''
model_clip = CLIPModel.from_pretrained("./openai/clip-vit-base-patch32")
processor_clip = AutoProcessor.from_pretrained("./openai/clip-vit-base-patch32")
tokenizer =  AutoTokenizer.from_pretrained("./openai/clip-vit-base-patch32")

with torch.no_grad():
    input_ids=tokenizer(captions_from_brain,return_tensors="pt",padding=True)
    embedding_brain= model_clip.get_text_features(**input_ids)

    input_ids=tokenizer(test_captions,return_tensors="pt",padding=True)
    embedding_captions= model_clip.get_text_features(**input_ids)

    input_ids=tokenizer(captions_from_images,return_tensors="pt",padding=True)
    embedding_images= model_clip.get_text_features(**input_ids)

clip_sim_brain_img=util.pytorch_cos_sim(embedding_brain, embedding_images).cpu()
clip_sim_brain_cap=util.pytorch_cos_sim(embedding_brain, embedding_captions).cpu()
clip_sim_img_cap=util.pytorch_cos_sim(embedding_images, embedding_captions).cpu()

# relative_brain_image_clip=clip_sim_brain_img.diag().mean()/clip_sim_img_cap.diag().mean()

print(f"[GROUND] CLIP Similarity GIT from image vs human: {clip_sim_img_cap.diag().mean()}")
print(f"[ABSOLUTE] CLIP Similarity GIT from brain vs image: {clip_sim_brain_img.diag().mean()}")
# print(f"[RELATIVE] CLIP Similarity   {relative_brain_image_clip.mean()}")

[GROUND] CLIP Similarity GIT from image vs human: 0.8316735029220581
[ABSOLUTE] CLIP Similarity GIT from brain vs image: 0.705390989780426
